In [8]:
import requests
import json
from pandas import DataFrame
import os

In [9]:
urlFmt = "https://dapi.kakao.com/v2/search/image?query={query}&page={page}&size={size}"

In [10]:
query = "봄 패션"
page = 1
size = 80
key = "899a1eedbcb6713b4e78b75f1e3ed026"
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

In [11]:
session = requests.Session()
session.headers.update({
    "Authorization" : "KakaoAK %s" % key
})

In [15]:
# 다음페이지가 존재하는 지 여부 --> 이 값이 false인 동안은 반복
isEnd = False
# 1페이지부터 시작
page = 1
# 저장된 이미지 카운트
count = 0

while not isEnd:
    #-----------------------------------------------------------------
    # 이전 예제 코드 시작
    url = urlFmt.format(query=query, page=page, size=size)
    # url
    #출력된 데이터의 변수값 확인
    r = session.get(url)
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
    r.encoding = "utf-8"
    data = json.loads(r.text)
    # data
    # -------------------------------------------------------------------
    # 이전 예제 코드 끝 
    # 추출한 리슽으를 미리 준비한 빈 리스트에  추가
    searchList = data['documents']

    # 증감식에 해당하는 isend변수 추출
    isEnd = bool(data['meta']['is_end'])
    page += 1
    # print("isEnd=", isEnd, ", 다음페이지=", page)

    # 이미지가 저장될 폴더 생성
    if os.path.exists(query) == False:
        os.mkdir(query) # 경로에 디렉토리 형성하기
    
    # 이미지저장
    for i, v in enumerate(searchList):
        count += 1
        docUrl = v['doc_url']
        imageUrl = v['image_url']
        # print(imageUrl)
        session.headers.update({
            "Authorization": "KakaoAK %s" % key,
            "Referer": docUrl,
            "User-Agent": userAgent
        })
    # 다운로드에 실패한 경우 다음 이미지로 이동
    if r.status_code != 200:
        continue
    
    fileName = "%s/%d05d.png" % (query, count)

    with open(fileName, 'wb') as f: 
        # with open('변수','wb') 바이너리(2진법으로) 쓰기모드
        # 'wb'모드로 파일을 open할 경우 raw date를 쓸수 있고 
        # 'rb'모드로 open할 경우 raw date로 읽을 수 있다. 
        f.write(r.raw.read())
        # r(읽기),w(쓰기),a(추가)
        # print(fileName, "(이)가 저장되었습니다.")
        # 읽어 들인 데이터를 저장한다.
print("종료")


종료
